In [1]:
import sqlalchemy as sa
import pandas as pd
import requests, json
%pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 17.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
my_db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'       : '5432',
    'username'   : 'kevli88',
    'password'   : '2023!Schulich',
    'database'   : 'kevli88_db'
}

In [3]:
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username   = my_db_secret['username'],
    password   = my_db_secret['password'],
    host       = my_db_secret['host'],
    port       = my_db_secret['port'],
    database   = my_db_secret['database']
)

In [4]:
my_db_engine = sa.create_engine(my_db_url)

In [5]:
with my_db_engine.connect() as connection:
    connection.execute('CREATE SCHEMA IF NOT EXISTS uploads;')

In [7]:
# Weather Data from OpenWeatherMap API
# write a function to get the weather data from OpenWeatherMap API
# use the API to get the current weather data for each city
# parse the JSON data from the API response
# collect the following data from the JSON file and store it into a DataFrame:
# City, Country, Date, Lat, Lng, Max Temp, Humidity, Cloudiness, Wind Speed
import pandas as pd
import requests
from datetime import datetime
import schedule
import time

In [10]:
import requests
import pandas as pd
from datetime import datetime

def get_weather_data(api_key, cities):
    base_url = "http://api.openweathermap.org/data/2.5/weather"
    columns = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
    current_weather = pd.DataFrame(columns=columns)

    for city in cities:
        params = {
            'q': city,
            'appid': api_key,
            'units': 'metric'
        }
        response = requests.get(base_url, params=params)
        data = response.json()

        if response.status_code == 200:
            current_weather = current_weather.append({
                "City": data['name'],
                "Country": data['sys']['country'],
                "Date": datetime.utcfromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S'),
                "Lat": data['coord']['lat'],
                "Lng": data['coord']['lon'],
                "Max Temp": data['main']['temp_max'],
                "Humidity": data['main']['humidity'],
                "Cloudiness": data['clouds']['all'],
                "Wind Speed": data['wind']['speed']
            }, ignore_index=True)
        else:
            print(f"Failed to retrieve data for {city}. Reason: {data['message']}")
    return current_weather

def get_forecast_data(lat, lon, api_key):
    units = 'metric'
    api_url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={api_key}&units={units}'
    response = requests.get(api_url)
    data = response.json()
    processed_data = []
    for d in data['list']:
        Adate = d['dt_txt'].split(' ')[0].replace('-','')
        url = f'https://dd.weather.gc.ca/alerts/cap/{Adate}'
        records = [Adate, d['main']['feels_like'], d['main']['temp_min'], d['main']['temp_max'], d['weather'][0]['description'], url]
        processed_data.append(records)

    dates = list(set([rec[0] for rec in processed_data]))

    final_result = []
    for d in dates:
        daily_records = [rec for rec in processed_data if rec[0] == d]
        feels_like_avg = round(sum([rec[1] for rec in daily_records]) / len(daily_records), 2)
        temp_min = min([rec[2] for rec in daily_records])
        temp_max = max([rec[3] for rec in daily_records])
        final_result.append([d, feels_like_avg, temp_min, temp_max, daily_records[0][4], daily_records[0][5]])
    return final_result

api_key = "6b7c918fb32224c10fecc2a16cc92d4a"
cities = ["London", "New York", "Toronto", "Vancouver", "Tokyo"]

# Get current weather data for specified cities
current_weather = get_weather_data(api_key, cities)
print(current_weather)

# For this example, let's use the latitude and longitude of Toronto (from your example) to get the forecast data
lat, lon = 43.6532, -79.3832
forecast_data = get_forecast_data(lat, lon, api_key)

# Convert forecast data into a DataFrame for easier visualization
forecast_df = pd.DataFrame(forecast_data, columns=["Date", "Feels Like Avg", "Min Temp", "Max Temp", "Description", "URL"])
print(forecast_df)


/var/folders/fm/9pz_54hn7ynbrz17999wv77m0000gn/T/ipykernel_82913/3808653019.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  current_weather = current_weather.append({
/var/folders/fm/9pz_54hn7ynbrz17999wv77m0000gn/T/ipykernel_82913/3808653019.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  current_weather = current_weather.append({
/var/folders/fm/9pz_54hn7ynbrz17999wv77m0000gn/T/ipykernel_82913/3808653019.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  current_weather = current_weather.append({
/var/folders/fm/9pz_54hn7ynbrz17999wv77m0000gn/T/ipykernel_82913/3808653019.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 

        City Country                 Date      Lat       Lng  Max Temp  \
0     London      GB  2023-08-11 04:27:35  51.5085   -0.1257     19.59   
1   New York      US  2023-08-11 04:30:34  40.7143  -74.0060     21.98   
2    Toronto      CA  2023-08-11 04:33:13  43.7001  -79.4163     21.06   
3  Vancouver      CA  2023-08-11 04:29:43  49.2497 -123.1193     19.68   
4      Tokyo      JP  2023-08-11 04:29:04  35.6895  139.6917     35.86   

  Humidity Cloudiness  Wind Speed  
0       88         75        4.12  
1       91         40        5.14  
2       63         75        5.66  
3       80         20        2.06  
4       53         40        7.72  
       Date  Feels Like Avg  Min Temp  Max Temp       Description  \
0  20230814           20.70     19.50     23.13   overcast clouds   
1  20230812           23.52     20.22     28.18     broken clouds   
2  20230811           21.73     17.22     26.31     broken clouds   
3  20230816           19.73     19.11     21.31  scattered clou